# Data Preparation

Splits Data into a global Training and 

Save to DataSet "Training_Battles"

In [1]:
import sklearn
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from kedro.extras.datasets.pandas import CSVDataSet

### Loading Data

In [2]:
catalog = context.catalog

In [3]:
Battle_Results = catalog.load('Battle_Results')
Weakness_Pokemon = catalog.load('Weakness_Pokemon')
AllPokemons = catalog.load('All_Pokemons')

2020-05-21 17:12:23,585 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results` (CSVDataSet)...
2020-05-21 17:12:28,518 - kedro.io.data_catalog - INFO - Loading data from `Weakness_Pokemon` (CSVDataSet)...
2020-05-21 17:12:28,526 - kedro.io.data_catalog - INFO - Loading data from `All_Pokemons` (CSVDataSet)...


### Split Data & Save Test Set

In [69]:
train_val_set, test_set = train_test_split(Battle_Results, test_size=0.1, random_state=42)

In [70]:
catalog.add("Battle_Results_Test", CSVDataSet(filepath="../../data/02_intermediate/Battle_Results_Test.csv"))
catalog.save("Battle_Results_Test", test_set)

2020-05-21 20:53:32,623 - kedro.io.data_catalog - INFO - Saving data to `Battle_Results_Test` (CSVDataSet)...


### Preprocess Dataset

In [175]:
def fetch_type_1(entry):
    return AllPokemons.loc[AllPokemons["Name"]==entry].Type_1.iloc[0]
def fetch_type_2(entry):
    return AllPokemons.loc[AllPokemons["Name"]==entry].Type_2.iloc[0]
def fetch_advantage(name1, name2):
    type1 = AllPokemons.loc[AllPokemons["Name"]==name1].Type_1.iloc[0]
    type2 = AllPokemons.loc[AllPokemons["Name"]==name2].Type_1.iloc[0]
    print(type1, type2)
    return Weakness_Pokemon.loc[Weakness_Pokemon["Types"]==type1, type2].iat[0]

In [176]:
def raw_to_relevant(dataset=train_val_set):
    # Delete Prices
    relevant = dataset.drop(columns=["Price_1", "Price_2"])
    # Insert Types
    # relevant.insert(loc=0, column="Type_1_1", value=relevant.Name_1.map(lambda name: fetch_type_1(name)))
    
    # Insert Advantages
    #relevant.insert(loc=4, column="Adv_1", value=relevant.apply(lambda row: fetch_advantage(row["Type_1_1"], row["Type_1_2"]), axis=1))
    #relevant.insert(loc=5, column="Adv_2", value=relevant.apply(lambda row: fetch_advantage(row["Type_1_2"], row["Type_1_1"]), axis=1))

    relevant.insert(loc=0, column="Adv_1", value=relevant.apply(lambda row: fetch_advantage(row["Name_1"], row["Name_2"]), axis=1))
    relevant.insert(loc=1, column="Adv_2", value=relevant.apply(lambda row: fetch_advantage(row["Name_2"], row["Name_1"]), axis=1))
    
    relevant.drop(columns=["Name_1", "Name_2"], inplace=True)
    return relevant

In [177]:
raw_to_relevant(data)

Water Bug
Fire Electric
Dragon Ground
Water Fire
Poison Water
Electric Electric
Normal Ground
Water Grass
Water Grass
Grass Ground
Bug Water
Electric Fire
Ground Dragon
Fire Water
Water Poison
Electric Electric
Ground Normal
Grass Water
Grass Water
Ground Grass


,Adv_1,Adv_2,Level_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,Level_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
2228072,1.0,1.0,8,68,65,59,82,63,69,False,75,336,192,172,103,103,279,False,Unknown,-336
611280,1.0,1.0,86,691,707,492,571,571,752,False,72,607,523,499,722,523,577,True,Sunshine,133
228854,1.0,1.0,93,636,698,484,538,538,538,False,73,446,529,720,328,529,284,False,Windy,324
2347836,20.0,0.6,6,41,47,85,85,29,72,False,86,919,886,797,1088,750,797,True,Night,-919
1920829,1.0,1.0,78,571,519,449,400,481,567,False,96,637,1178,1049,456,456,682,False,Unknown,-560
283919,0.6,0.6,92,517,299,615,828,481,396,False,87,882,762,729,1051,762,842,True,Windy,-352
1421838,1.0,1.0,59,286,241,292,348,317,164,False,10,72,100,113,27,39,54,False,Rain,286
2394630,0.6,6.0,29,156,170,117,117,146,126,False,22,139,71,141,104,78,71,False,Windy,-22
2398651,0.6,6.0,91,893,845,845,626,813,626,False,19,115,85,178,154,60,93,False,Night,893
950166,6.0,0.6,25,118,134,59,124,49,68,False,80,539,384,714,307,384,272,False,Night,-539


In [173]:
data = train_val_set.iloc[:10]; data

,Name_1,Level_1,Price_1,HP_1,Attack_1,Defense_1,Sp_Atk_1,Sp_Def_1,Speed_1,Legendary_1,...,Price_2,HP_2,Attack_2,Defense_2,Sp_Atk_2,Sp_Def_2,Speed_2,Legendary_2,WeatherAndTime,BattleResult
2228072,Psyduck,8,406,68,65,59,82,63,69,False,...,1185,336,192,172,103,103,279,False,Unknown,-336
611280,Rapidash,86,3784,691,707,492,571,571,752,False,...,3451,607,523,499,722,523,577,True,Sunshine,133
228854,Dratini,93,3432,636,698,484,538,538,538,False,...,2836,446,529,720,328,529,284,False,Windy,324
2347836,Horsea,6,359,41,47,85,85,29,72,False,...,5237,919,886,797,1088,750,797,True,Night,-919
1920829,Golbat,78,2987,571,519,449,400,481,567,False,...,4458,637,1178,1049,456,456,682,False,Unknown,-560
283919,Magnemite,92,3136,517,299,615,828,481,396,False,...,5028,882,762,729,1051,762,842,True,Windy,-352
1421838,Porygon,59,1648,286,241,292,348,317,164,False,...,405,72,100,113,27,39,54,False,Rain,286
2394630,Seaking,29,832,156,170,117,117,146,126,False,...,604,139,71,141,104,78,71,False,Windy,-22
2398651,Poliwrath,91,4648,893,845,845,626,813,626,False,...,685,115,85,178,154,60,93,False,Night,893
950166,Bellsprout,25,552,118,134,59,124,49,68,False,...,2600,539,384,714,307,384,272,False,Night,-539


In [140]:
data.insert(loc=4, column="Adv_1", value=data.apply(lambda row: fetch_advantage(row["Type_1_1"], row["Type_1_2"]), axis=1))

KeyError: ('Type_1_1', 'occurred at index 2228072')

In [106]:
data.insert(loc=2, column="Type_1_3", value=data.map(lambda x: fetch(x.Name_1)))

AttributeError: 'DataFrame' object has no attribute 'map'

In [142]:
fetch_advantage(data.iloc[0]["Type_1_1"], data.iloc[0]["Type_1_1"])

KeyError: 'Type_1_1'

In [156]:
data.apply(lambda row: fetch_advantage(row["Type_1_1"], row["Type_1_1"]), axis=1)

KeyError: ('Type_1_1', 'occurred at index 2228072')

In [153]:
Weakness_Pokemon.loc[Weakness_Pokemon["Types"]=="Fire", "Grass"].iat[0]

50.0

In [171]:
AllPokemons["Type_1"].isnull().sum()

0

### Notes
Target:
- BattleResults, reformated to range in [-1, 1], where 0.5 means the first pokemons wins with 50% hp left.
Creates a unified DataFile out of Battles, Weaknesses, and Types

- remove field Name_1, Name_2
- remove field Price_1, Price_2
- Add fields, Adv_1, Adv_2

Save to DataSet "Training_Battles"
Possible later changes:
- Secondary types for simplicity.
- Try to include second nature
- Analyze which features correlate, the most, speed might be very important?
- Analyze Stat Differences?, with mean speed, mean attack?d